In [25]:
%%capture
!pip install datasets

# Random Forest

In [4]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords

In [8]:
from datasets import load_dataset
train_data, test_data = load_dataset("trec", split =['train', 'test'])

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
X = train_data['text'] + test_data['text']
y = train_data['fine_label'] + test_data['fine_label']

In [10]:
train_num = len(train_data)
test_num = len(test_data)

In [11]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer1 = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer1.fit_transform(documents).toarray()

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter1 = TfidfTransformer()
X = tfidfconverter1.fit_transform(X).toarray()

In [14]:
X_train, X_test = X[:train_num], X[train_num:]
y_train, y_test = y[:train_num], y[train_num:]

In [15]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=1000, random_state=0)

In [16]:
y_pred = classifier.predict(X_test)

In [18]:
accuracy_score(y_test, y_pred)

0.676

In [31]:
# Display F1 score
f1_score(y_test, y_pred, average='macro')

0.5526325784529024